In [53]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib
import os
import warnings
import csv
from sklearn.linear_model import LassoCV
from stg import STG
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm


In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Suppress all warnings
warnings.filterwarnings('ignore')


In [55]:
model_used = 'Neural_Network'
print_interval = 5000
epochs = 10000
selection_method = 'Stochastic Gates'
# selection_method = 'lasso'

Light_GBM_global = [
    # "flair_t1ce_t2",
    # "flair_t1ce_t2",
    # "flair",
    # "flair_t1ce",
    "flair_t1ce"
]
Light_GBM_local = [
    # "flair_t1ce_t2",
    # "flair_t1ce",
    # "flair",
    # "t2",
    "flair_t1ce",
]


In [56]:
def ensure_directory_exists(filepath):
    os.makedirs(os.path.dirname(filepath), exist_ok=True)


In [57]:
def load_radiomic_fetures(target_directory, file_name):
    # Load the numpy array from the file in the target directory
    file_path = os.path.join(target_directory, file_name)
    array = np.load(file_path)
    print(f"Array loaded from '{file_path}'")
    return array

In [58]:
def load_model_and_parameters(modality_used,  model_class):
    model_dir = f"./models/{selection_method}_feature_selection/{model_used}/{modality_used}/"
    
    model_file = os.path.join(model_dir, 'model.pt')
    params_file = os.path.join(model_dir, 'params.txt')
    mask_file = os.path.join(model_dir, f'{selection_method}_mask.npy')
    # Load the selected features mask
    mask = np.load(mask_file)
    inputdim = np.count_nonzero(mask)
    # Load the model
    model = model_class(inputdim)
    model.load_state_dict(torch.load(model_file))
    model.eval()  # Set the model to evaluation mode
    
    # Load the parameters
    with open(params_file, 'r') as file:
        params = file.read().strip()
    
    
    print(f"Model, parameters, and {selection_method} mask loaded successfully for modality {modality_used}.")
    
    return model, params, mask

In [59]:
def save_model_and_parameters(model, modality_used, mask, params):
    model_dir = f"./models/{selection_method}_feature_selection/{model_used}_256_dim/{modality_used}/Intersection_mask_1_2/"
    ensure_directory_exists(model_dir)
    model_file = os.path.join(model_dir, 'model.pt')
    params_file = os.path.join(model_dir, 'params.txt')
    mask_file = os.path.join(model_dir, f'{selection_method}_mask.npy')
    
    # Save the model
    torch.save(model.state_dict(), model_file)
    
    # Save the parameters
    with open(params_file, 'w') as file:
        file.write(f"Best parameters: {params}\n")
    
    # Save the selected features mask
    np.save(mask_file, mask)
    
    print(f"Model, parameters, and {selection_method} mask saved successfully for modality {modality_used}.")


In [60]:
def load_and_combine_features(modality_keys, dataset_type):
    combined_features = []
    for modality in modality_keys:
        # Load the features for each modality
        features = np.load(f'../local_spatial_Framework/features/{modality}/{dataset_type}/{dataset_type}_backbone_outputs.npy')
        combined_features.append(features)
    # Combine features along the feature dimension (axis=1)
    return np.concatenate(combined_features, axis=1)


In [61]:

def make_csv(y_pred_validation, modality_used):
    df = pd.read_csv('../dataset/MICCAI_BraTS2020_ValidationData/survival_evaluation.csv')
    validation_ids = df['BraTS20ID'].values
    
    #removing 116 paitents ID from list
    validation_ids = np.delete(validation_ids, 26, axis=0)
    
    filename = f"../radiomics_local_global_predictions/{selection_method}_feture_selection/{model_used}_256_dim/Intersection_mask_1_2/{modality_used}_{model_used}.csv"

    ensure_directory_exists(filename)

    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["ID", "Days"])
        for id, day in zip(validation_ids, y_pred_validation):
            writer.writerow([id, day])

    print(f"CSV file '{filename}' created successfully.")


In [62]:
def load_features(modality_used):
    base_dir = os.path.join('../Global_extracted_features', modality_used)
    train_features = np.load(os.path.join(base_dir, 'train_features.npy'))
    validate_features = np.load(os.path.join(base_dir, 'validate_features.npy'))
    train_labels = np.load(os.path.join(base_dir, 'train_labels.npy'))
    return train_features, validate_features, train_labels


In [63]:
# # 1st model
# class SimpleNN_1(nn.Module):
#     def __init__(self, input_dim):
#         super(SimpleNN_1, self).__init__()
#         self.fc1 = nn.Linear(input_dim, 200)
#         self.fc2 = nn.Linear(200, 50)
#         self.fc3 = nn.Linear(50, 1)

#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


In [64]:
# # 2nd model
# class SimpleNN(nn.Module):
#     def __init__(self, input_dim):
#         super(SimpleNN, self).__init__()
#         self.fc1 = nn.Linear(input_dim, 128)
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, 1)

#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


In [65]:
# 3rd model
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [66]:

def train_model(train_features, test_features, train_labels, modality_used):
    scaler = StandardScaler()
    train_features_scaled = scaler.fit_transform(train_features)
    test_features_scaled = scaler.transform(test_features)
    
    # Print feature size before Lasso feature selection
    # print(f"Size of features before Lasso: {train_features_scaled.shape}")

    # # Lasso Feature Selection with increased regularization
    # lasso = LassoCV(cv=5, random_state=42, max_iter=100, alphas=np.logspace(-4, -0.5, 30)).fit(train_features_scaled, train_labels)
    
    # # Select non-zero coefficients
    # mask = lasso.coef_ != 0
    model_dir = f"./models/{selection_method}_feature_selection/{model_used}/{modality_used}/"
    mask_file = os.path.join(model_dir, f'{selection_method}_mask.npy')
    mask = np.load(mask_file)

    train_features_selected = train_features_scaled[:, mask]
    test_features_selected = test_features_scaled[:, mask]

    print(f"Size of features after lasso Gates: {train_features_selected.shape}")
      
    # Split the selected features into training and validation sets
    train_features_final, val_features_final, train_labels_final, val_labels_final = train_test_split(
        train_features_selected, train_labels, test_size=0.2, random_state=42)

    # Convert features and labels to PyTorch tensors
    train_features_tensor = torch.tensor(train_features_final, dtype=torch.float32).to(device)
    train_labels_tensor = torch.tensor(train_labels_final, dtype=torch.float32).view(-1, 1).to(device)
    val_features_tensor = torch.tensor(val_features_final, dtype=torch.float32).to(device)
    val_labels_tensor = torch.tensor(val_labels_final, dtype=torch.float32).view(-1, 1).to(device)
    test_features_tensor = torch.tensor(test_features_selected, dtype=torch.float32).to(device)

    # Create DataLoader for training data
    train_dataset = TensorDataset(train_features_tensor, train_labels_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Initialize and train the neural network
    input_dim = train_features_selected.shape[1]
    model = SimpleNN(input_dim).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    best_model = None
    best_accuracy = float('inf')

    model.train()
    for epoch in tqdm(range(epochs), desc="Training Epochs"):  # Adjust the number of epochs as needed
        epoch_loss = 0
        for batch_features, batch_labels in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        
        if (epoch + 1) % print_interval == 0:
            print(f'Epoch {epoch + 1}, Loss: {epoch_loss / len(train_loader)}')

        # Evaluate on the validation set
        model.eval()
        with torch.no_grad():
            val_outputs = model(val_features_tensor)
            val_loss = criterion(val_outputs, val_labels_tensor).item()


        # Check for the best model
        if val_loss < best_accuracy:
            best_accuracy = val_loss
            best_model = model.state_dict()

        if (epoch + 1) % print_interval == 0:
            print(f'Epoch {epoch + 1}, Validation Loss: {val_loss}, best: {best_accuracy}')
        model.train()

    # Save the best model
    model.load_state_dict(best_model)
    save_model_and_parameters(model, modality_used, mask, {f"epochs": {epochs}, "lr": 0.001})

    # Make predictions on the test set
    model.eval()
    with torch.no_grad():
        y_pred_test = model(test_features_tensor).cpu().numpy().flatten()

    make_csv(y_pred_test, modality_used)

In [67]:
def train_model_with_stochastic_gates(train_features, test_features, train_labels, modality_used, device='cpu', validation_split=0.2):
    # Split training data into train and validation sets for STG
    train_features_stg, val_features_stg, train_labels_stg, val_labels_stg = train_test_split(
        train_features, train_labels, test_size=validation_split, random_state=42)

    scaler = StandardScaler()
    train_features_scaled = scaler.fit_transform(train_features_stg)
    val_features_scaled = scaler.transform(val_features_stg)
    test_features_scaled = scaler.transform(test_features)
    
    # Ensure data is in numpy array format
    train_features_scaled = np.asarray(train_features_scaled)
    val_features_scaled = np.asarray(val_features_scaled)
    train_labels_stg = np.asarray(train_labels_stg)
    val_labels_stg = np.asarray(val_labels_stg)
    test_features_scaled = np.asarray(test_features_scaled)
    
    print(f"Size of training features before Stochastic Gates: {train_features_scaled.shape}")
    print(f"Size of validation features before Stochastic Gates: {val_features_scaled.shape}")

    # # Define and train the STG model
    # stg = STG(task_type='regression', input_dim=train_features_scaled.shape[1], output_dim=1,
    #           hidden_dims=[100, 50], activation='relu', optimizer='Adam', learning_rate=0.01,
    #           batch_size=32, feature_selection=True, device=device)
    
    # try:
    #     # Fit the STG model with train data and validate on the validation set
    #     stg.fit(train_features_scaled, train_labels_stg, nr_epochs=10000, verbose=1, print_interval=5000,
    #             valid_X=val_features_scaled, valid_y=val_labels_stg)
    # except Exception as e:
    #     print(f"Error during STG fitting: {str(e)}")
    #     print(f"Train features shape: {train_features_scaled.shape}")
    #     print(f"Train labels shape: {train_labels_stg.shape}")
    #     print(f"Validation features shape: {val_features_scaled.shape}")
    #     print(f"Validation labels shape: {val_labels_stg.shape}")
    #     return

    # Get feature importance scores using get_gates method
    # try:
    #     importance_scores = stg.get_gates(mode='prob') 
    # except Exception as e:
    #     print(f"Error getting feature importance: {str(e)}")
    #     print("Using all features.")
    #     importance_scores = np.ones(train_features_scaled.shape[1])

    # # Select features with importance scores above a threshold
    # mask = importance_scores > 0.5  # Adjust threshold as necessary
    # t1, t2, mask = load_model_and_parameters(modality_used, SimpleNN_1)
    model_dir = f"./models/{selection_method}_feature_selection/Intersection_mask.npy"
    mask = np.load(model_dir)
    
    # Apply feature selection to the full training set and test set
    train_features_selected = scaler.fit_transform(train_features)[:, mask]
    test_features_selected = scaler.transform(test_features)[:, mask]
    print(f"Size of features after Stochastic Gates: {train_features_selected.shape}")
    
   # Split the selected features into training and validation sets
    train_features_final, val_features_final, train_labels_final, val_labels_final = train_test_split(
        train_features_selected, train_labels, test_size=0.2, random_state=42)

    # Convert features and labels to PyTorch tensors
    train_features_tensor = torch.tensor(train_features_final, dtype=torch.float32).to(device)
    train_labels_tensor = torch.tensor(train_labels_final, dtype=torch.float32).view(-1, 1).to(device)
    val_features_tensor = torch.tensor(val_features_final, dtype=torch.float32).to(device)
    val_labels_tensor = torch.tensor(val_labels_final, dtype=torch.float32).view(-1, 1).to(device)
    test_features_tensor = torch.tensor(test_features_selected, dtype=torch.float32).to(device)

    # Create DataLoader for training data
    train_dataset = TensorDataset(train_features_tensor, train_labels_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    # Initialize and train the neural network
    input_dim = train_features_selected.shape[1]
    model = SimpleNN(input_dim).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    best_model = None
    best_accuracy = float('inf')

    model.train()
    for epoch in tqdm(range(epochs), desc="Training Epochs"):  # Adjust the number of epochs as needed
        epoch_loss = 0
        for batch_features, batch_labels in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        
        if (epoch + 1) % print_interval == 0:
            print(f'Epoch {epoch + 1}, Loss: {epoch_loss / len(train_loader)}')

        # Evaluate on the validation set
        model.eval()
        with torch.no_grad():
            val_outputs = model(val_features_tensor)
            val_loss = criterion(val_outputs, val_labels_tensor).item()


        # Check for the best model
        if val_loss < best_accuracy:
            best_accuracy = val_loss
            best_model = model.state_dict()

        if (epoch + 1) % print_interval == 0:
            print(f'> Epoch {epoch + 1}, Validation Loss: {val_loss}, best: {best_accuracy}')
        model.train()

    # Save the best model
    model.load_state_dict(best_model)
    save_model_and_parameters(model, modality_used, mask, {f"epochs": {epochs}, "lr": 0.001})

    # Make predictions on the test set
    model.eval()
    with torch.no_grad():
        y_pred_test = model(test_features_tensor).cpu().numpy().flatten()

    make_csv(y_pred_test, modality_used)

In [68]:

radiomic_Train_fetures = load_radiomic_fetures("../radiomics features/all", "radiomics_train.npy")
radiomic_Validation_fetures = load_radiomic_fetures("../radiomics features/all", "radiomics_validate.npy")

print("radiomic_Train_fetures",radiomic_Train_fetures.shape)
print("radiomic_Validation_fetures",radiomic_Validation_fetures.shape)

Array loaded from '../radiomics features/all/radiomics_train.npy'
Array loaded from '../radiomics features/all/radiomics_validate.npy'
radiomic_Train_fetures (235, 400)
radiomic_Validation_fetures (29, 400)


In [69]:
for modality_used_global, modality_used_local in zip(Light_GBM_global, Light_GBM_local):
    modality_key_local = modality_used_local.split("_")
    modality_keys_list_global_features = modality_used_global.split("_")
    print(f"\nLoading and combining features... \n local-{modality_used_local}\n global-{modality_used_global}")

    local_train_features = load_and_combine_features(modality_key_local, 'train')
    local_validation_features = load_and_combine_features(modality_key_local, 'validation')
    
    global_train_features, global_validate_features, train_labels = load_features(modality_used_global)
    
    print("global_train_features",global_train_features.shape, "local_train_features", local_train_features.shape)
    print("global_validate_features", global_validate_features.shape, "local_validation_features",local_validation_features.shape)


    local_global_training_features = np.concatenate((global_train_features, local_train_features), axis=1)
    local_global_validation_features = np.concatenate((global_validate_features, local_validation_features), axis=1)
    
#   To remove the data of absent paitents from the local and global features
    train_labels = np.delete(train_labels, 98, axis=0)
    local_global_training_features = np.delete(local_global_training_features, 98, axis=0)
    # local_global_validation_features = np.delete(local_global_validation_features, 26, axis=0)


    training_all_features = np.concatenate((local_global_training_features, radiomic_Train_fetures), axis=1)
    validation_all_features = np.concatenate((local_global_validation_features,radiomic_Validation_fetures), axis=1)

    print("combining all")
    print("radiomics_local_global_training",training_all_features.shape)
    print("radiomics_local_global_Validation",validation_all_features.shape)

    modality_used = 'global_' + modality_used_global + '___local_' + modality_used_local
    if selection_method == "Stochastic Gates":
        train_model_with_stochastic_gates(training_all_features, validation_all_features, train_labels, modality_used)
    elif selection_method == "lasso":
        train_model(training_all_features, validation_all_features, train_labels, modality_used)
    print("\n-----------------------------------------------------------------------------------------------------------\n")


Loading and combining features... 
 local-flair_t1ce
 global-flair_t1ce
global_train_features (236, 744) local_train_features (236, 128)
global_validate_features (29, 744) local_validation_features (29, 128)
combining all
radiomics_local_global_training (235, 1272)
radiomics_local_global_Validation (29, 1272)
Size of training features before Stochastic Gates: (188, 1272)
Size of validation features before Stochastic Gates: (47, 1272)
Size of features after Stochastic Gates: (235, 392)


Training Epochs:   0%|          | 8/10000 [00:00<02:12, 75.27it/s]

Training Epochs:  50%|█████     | 5012/10000 [01:14<01:10, 70.73it/s]

Epoch 5000, Loss: 4.6488662919230705e-10
> Epoch 5000, Validation Loss: 250768.484375, best: 141542.359375


Training Epochs: 100%|██████████| 10000/10000 [02:32<00:00, 65.57it/s]


Epoch 10000, Loss: 1.0051192285883321e-09
> Epoch 10000, Validation Loss: 236545.0625, best: 141542.359375
Model, parameters, and Stochastic Gates mask saved successfully for modality global_flair_t1ce___local_flair_t1ce.
CSV file '../radiomics_local_global_predictions/Stochastic Gates_feture_selection/Neural_Network_256_dim/Intersection_mask_1_2/global_flair_t1ce___local_flair_t1ce_Neural_Network.csv' created successfully.

-----------------------------------------------------------------------------------------------------------



## For stocastic Gates

Loading and combining features... 
 local-flair_t1ce_t2
 global-flair_t1ce_t2
global_train_features (236, 744) local_train_features (236, 192)
global_validate_features (29, 744) local_validation_features (29, 192)
combining all
radiomics_local_global_training (235, 1336)
radiomics_local_global_Validation (28, 1336)
Size of training features before Stochastic Gates: (188, 1336)
Size of validation features before Stochastic Gates: (47, 1336)
Epoch: 5000: loss=123142.164062 valid_loss=159916.453125
Epoch: 10000: loss=123142.691406 valid_loss=160539.062500
Size of features after Stochastic Gates: (235, 501)
Epoch 500, Loss: 1.2582521438598633
Epoch 1000, Loss: 54.035579800605774
CSV file '../radiomics_local_global_predictions/Stochastic Gates_feture_selection/Neural_Network/global_flair_t1ce_t2___local_flair_t1ce_t2_Neural_Network.csv' created successfully.
Model, parameters, and Stochastic Gates mask saved successfully for modality global_flair_t1ce_t2___local_flair_t1ce_t2.

-----------------------------------------------------------------------------------------------------------


Loading and combining features... 
 local-flair_t1ce
 global-flair_t1ce_t2
global_train_features (236, 744) local_train_features (236, 128)
global_validate_features (29, 744) local_validation_features (29, 128)
combining all
radiomics_local_global_training (235, 1272)
radiomics_local_global_Validation (28, 1272)
Size of training features before Stochastic Gates: (188, 1272)
Size of validation features before Stochastic Gates: (47, 1272)
Epoch: 5000: loss=123166.786458 valid_loss=160644.562500
Epoch: 10000: loss=123162.050781 valid_loss=161601.531250
Size of features after Stochastic Gates: (235, 485)
Epoch 500, Loss: 0.05665457126451656
Epoch 1000, Loss: 81.00391387939453
CSV file '../radiomics_local_global_predictions/Stochastic Gates_feture_selection/Neural_Network/global_flair_t1ce_t2___local_flair_t1ce_Neural_Network.csv' created successfully.
Model, parameters, and Stochastic Gates mask saved successfully for modality global_flair_t1ce_t2___local_flair_t1ce.

-----------------------------------------------------------------------------------------------------------


Loading and combining features... 
 local-flair
 global-flair
global_train_features (236, 744) local_train_features (236, 64)
global_validate_features (29, 744) local_validation_features (29, 64)
combining all
radiomics_local_global_training (235, 1208)
radiomics_local_global_Validation (28, 1208)
Size of training features before Stochastic Gates: (188, 1208)
Size of validation features before Stochastic Gates: (47, 1208)
Epoch: 5000: loss=123177.072917 valid_loss=159584.406250
Epoch: 10000: loss=123162.636719 valid_loss=159828.000000
Size of features after Stochastic Gates: (235, 454)
Epoch 500, Loss: 1.3944591656327248
Epoch 1000, Loss: 56.555126428604126
CSV file '../radiomics_local_global_predictions/Stochastic Gates_feture_selection/Neural_Network/global_flair___local_flair_Neural_Network.csv' created successfully.
Model, parameters, and Stochastic Gates mask saved successfully for modality global_flair___local_flair.

-----------------------------------------------------------------------------------------------------------


Loading and combining features... 
 local-t2
 global-flair_t1ce
global_train_features (236, 744) local_train_features (236, 64)
global_validate_features (29, 744) local_validation_features (29, 64)
combining all
radiomics_local_global_training (235, 1208)
radiomics_local_global_Validation (28, 1208)
Size of training features before Stochastic Gates: (188, 1208)
Size of validation features before Stochastic Gates: (47, 1208)
Epoch: 5000: loss=123142.777344 valid_loss=157602.421875
Epoch: 10000: loss=123144.131510 valid_loss=157510.921875
Size of features after Stochastic Gates: (235, 498)
Epoch 500, Loss: 0.5567352790385485
Epoch 1000, Loss: 56.83930325508118
CSV file '../radiomics_local_global_predictions/Stochastic Gates_feture_selection/Neural_Network/global_flair_t1ce___local_t2_Neural_Network.csv' created successfully.
Model, parameters, and Stochastic Gates mask saved successfully for modality global_flair_t1ce___local_t2.

-----------------------------------------------------------------------------------------------------------


Loading and combining features... 
 local-flair_t1ce
 global-flair_t1ce
global_train_features (236, 744) local_train_features (236, 128)
global_validate_features (29, 744) local_validation_features (29, 128)
combining all
radiomics_local_global_training (235, 1272)
radiomics_local_global_Validation (28, 1272)
Size of training features before Stochastic Gates: (188, 1272)
Size of validation features before Stochastic Gates: (47, 1272)
Epoch: 5000: loss=123157.269531 valid_loss=159203.859375
Epoch: 10000: loss=123160.901042 valid_loss=159248.937500
Size of features after Stochastic Gates: (235, 522)
Epoch 500, Loss: 48.375020027160645
Epoch 1000, Loss: 0.5991256311535835
CSV file '../radiomics_local_global_predictions/Stochastic Gates_feture_selection/Neural_Network/global_flair_t1ce___local_flair_t1ce_Neural_Network.csv' created successfully.
Model, parameters, and Stochastic Gates mask saved successfully for modality global_flair_t1ce___local_flair_t1ce.

-----------------------------------------------------------------------------------------------------------


# for Lasso 

Loading and combining features... 
 local-flair_t1ce_t2
 global-flair_t1ce_t2
global_train_features (236, 744) local_train_features (236, 192)
global_validate_features (29, 744) local_validation_features (29, 192)
combining all
radiomics_local_global_training (235, 1336)
radiomics_local_global_Validation (28, 1336)
Size of features before Lasso: (235, 1336)
Size of features after Stochastic Gates: (235, 670)
Epoch 500, Loss: 3.097682014107704
Epoch 1000, Loss: 211.7134804725647
CSV file '../radiomics_local_global_predictions/lasso_feture_selection/Neural_Network/global_flair_t1ce_t2___local_flair_t1ce_t2_Neural_Network.csv' created successfully.
Model, parameters, and lasso mask saved successfully for modality global_flair_t1ce_t2___local_flair_t1ce_t2.

-----------------------------------------------------------------------------------------------------------


Loading and combining features... 
 local-flair_t1ce
 global-flair_t1ce_t2
global_train_features (236, 744) local_train_features (236, 128)
global_validate_features (29, 744) local_validation_features (29, 128)
combining all
radiomics_local_global_training (235, 1272)
radiomics_local_global_Validation (28, 1272)
Size of features before Lasso: (235, 1272)
Size of features after Stochastic Gates: (235, 651)
Epoch 500, Loss: 0.02359358873218298
Epoch 1000, Loss: 3.981287568807602
CSV file '../radiomics_local_global_predictions/lasso_feture_selection/Neural_Network/global_flair_t1ce_t2___local_flair_t1ce_Neural_Network.csv' created successfully.
Model, parameters, and lasso mask saved successfully for modality global_flair_t1ce_t2___local_flair_t1ce.

-----------------------------------------------------------------------------------------------------------


Loading and combining features... 
 local-flair
 global-flair
global_train_features (236, 744) local_train_features (236, 64)
global_validate_features (29, 744) local_validation_features (29, 64)
combining all
radiomics_local_global_training (235, 1208)
radiomics_local_global_Validation (28, 1208)
Size of features before Lasso: (235, 1208)
Size of features after Stochastic Gates: (235, 606)
Epoch 500, Loss: 30.867851197719574
Epoch 1000, Loss: 251.14953899383545
CSV file '../radiomics_local_global_predictions/lasso_feture_selection/Neural_Network/global_flair___local_flair_Neural_Network.csv' created successfully.
Model, parameters, and lasso mask saved successfully for modality global_flair___local_flair.

-----------------------------------------------------------------------------------------------------------


Loading and combining features... 
 local-t2
 global-flair_t1ce
global_train_features (236, 744) local_train_features (236, 64)
global_validate_features (29, 744) local_validation_features (29, 64)
combining all
radiomics_local_global_training (235, 1208)
radiomics_local_global_Validation (28, 1208)
Size of features before Lasso: (235, 1208)
Size of features after Stochastic Gates: (235, 583)
Epoch 500, Loss: 3.6403137370944023
Epoch 1000, Loss: 1.103215642273426
CSV file '../radiomics_local_global_predictions/lasso_feture_selection/Neural_Network/global_flair_t1ce___local_t2_Neural_Network.csv' created successfully.
Model, parameters, and lasso mask saved successfully for modality global_flair_t1ce___local_t2.

-----------------------------------------------------------------------------------------------------------


Loading and combining features... 
 local-flair_t1ce
 global-flair_t1ce
global_train_features (236, 744) local_train_features (236, 128)
global_validate_features (29, 744) local_validation_features (29, 128)
combining all
radiomics_local_global_training (235, 1272)
radiomics_local_global_Validation (28, 1272)
Size of features before Lasso: (235, 1272)
Size of features after Stochastic Gates: (235, 621)
Epoch 500, Loss: 358.2224006652832
Epoch 1000, Loss: 3.599237322807312
CSV file '../radiomics_local_global_predictions/lasso_feture_selection/Neural_Network/global_flair_t1ce___local_flair_t1ce_Neural_Network.csv' created successfully.
Model, parameters, and lasso mask saved successfully for modality global_flair_t1ce___local_flair_t1ce.

-----------------------------------------------------------------------------------------------------------
